# Strom-Endverbrauch in der Schweiz
Dieses Notebook visualisiert den Strom-Endverbrauch in der Schweiz über die Jahre, basierend auf verschiedenen Sektoren.

## Daten laden, aufbereiten und anreichern
In diesem Abschnitt werden die Daten geladen, gefiltert und für die Visualisierung vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd
import plotly.express as px

# CSV-Daten laden
file_path = '../data/verbrauch/Strom_Endverbrauch_Schweiz.csv'
data = pd.read_csv(file_path, sep=';')

# Daten filtern: Nur Jahre zwischen 2015 und 2024
data = data[(data['Jahr'] >= 2015) & (data['Jahr'] <= 2024)]

# Spaltennamen anpassen: Entfernen von " in GWh"
data.columns = [col.replace(' in GWh', '') for col in data.columns]

# Daten überprüfen: Zeige die ersten Zeilen
data.head()

## Interaktives Diagramm erstellen

In [ ]:
# Interaktives Diagramm erstellen 
# Daten transformieren: Umwandeln in ein langes Format für die Visualisierung
data_melted = data.melt(id_vars=['Jahr'], var_name='Sektor', value_name='Verbrauch (GWh)')

# Top 3 Sektoren basierend auf dem Gesamtverbrauch berechnen
top_sectors = data_melted.groupby('Sektor')['Verbrauch (GWh)'].sum().nlargest(3).index

# Farben für die Top 3 Sektoren definieren, andere Sektoren in Grau darstellen
color_map = {
    sector: color for sector, color in zip(top_sectors, ['#636efa', '#EF553B', '#00cc96'])
}
color_map.update({sector: 'lightgray' for sector in data_melted['Sektor'].unique() if sector not in top_sectors})

# Diagramm erstellen mit farblicher Hervorhebung der Top 3 Sektoren
fig = px.line(
    data_melted,
    x='Jahr',
    y='Verbrauch (GWh)',
    color='Sektor',
    title='Strom-Endverbrauch in der Schweiz nach Sektoren',
    labels={'Jahr': 'Jahr', 'Verbrauch (GWh)': 'Verbrauch (GWh)', 'Sektor': 'Sektor'},
    color_discrete_map=color_map
)

# Linien und Marker anpassen
fig.update_traces(mode='markers+lines', marker=dict(size=6), line=dict(width=2))

# Layout anpassen: Einheitliche Achsen und klare Beschriftungen
fig.update_layout(
    xaxis=dict(tickmode='linear', title='Jahr'),
    yaxis=dict(title='Verbrauch (GWh)'),
    legend_title='Sektor',
    template='simple_white',  # Minimalistisches Design
    title=dict(font=dict(size=20))
)

# Diagramm als HTML speichern
fig.write_html('../docs/assets/diagramme/Strom_Endverbrauch_Schweiz.html')

# Diagramm anzeigen
fig.show()